# Image Prediction - Quick Start


In [1]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset

/Users/traveroliver/Library/Caches/pypoetry/virtualenvs/autoguluon-sandbox-uyisuGyl-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
train_dataset, _, test_dataset = ImageDataset.from_folders('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip')
print(train_dataset)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40895/40895 [01:19<00:00, 513.17KB/s]


data/
├── test/
└── train/
                                                 image  label
0    /Users/traveroliver/.gluoncv/datasets/shopee-i...      0
1    /Users/traveroliver/.gluoncv/datasets/shopee-i...      0
2    /Users/traveroliver/.gluoncv/datasets/shopee-i...      0
3    /Users/traveroliver/.gluoncv/datasets/shopee-i...      0
4    /Users/traveroliver/.gluoncv/datasets/shopee-i...      0
..                                                 ...    ...
795  /Users/traveroliver/.gluoncv/datasets/shopee-i...      3
796  /Users/traveroliver/.gluoncv/datasets/shopee-i...      3
797  /Users/traveroliver/.gluoncv/datasets/shopee-i...      3
798  /Users/traveroliver/.gluoncv/datasets/shopee-i...      3
799  /Users/traveroliver/.gluoncv/datasets/shopee-i...      3

[800 rows x 2 columns]


In [9]:
predictor = ImagePredictor()
# since the original dataset does not provide validation split, the `fit` function splits it randomly with 90/10 ratio
predictor.fit(train_dataset, hyperparameters={'epochs': 2})  # you can trust the default config, we reduce the # epoch to save some build time

ImagePredictor sets accuracy as default eval_metric for classification problems.
`time_limit=auto` set to `time_limit=7200`.
Reset labels to [0, 1, 2, 3]
Randomly split train_data into train[720]/validation[80] splits.
No GPU detected/allowed, using most conservative search space.
Starting fit without HPO
modified configs(<old> != <new>): {
root.misc.seed       42 != 463
root.misc.num_workers 4 != 8
root.img_cls.model   resnet101 != resnet18
root.train.epochs    200 != 2
root.train.batch_size 32 != 16
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.gpus            (0,) != ()
}
Saved config to /Users/traveroliver/workspace/autoguluon-sandbox/d0604d44/.trial_0/config.yaml
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/traveroliver/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
Model resnet18 created, param count:                                         11178564
AMP n

In [10]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.690, val acc: 0.900


In [11]:
image_path = test_dataset.iloc[0]['image']
result = predictor.predict(image_path)
print(result)

0    0
Name: label, dtype: int64


In [13]:
proba = predictor.predict_proba(image_path)
print(proba)

          0         1         2         3
0  0.557787  0.377065  0.058982  0.006166


In [14]:
bulk_result = predictor.predict(test_dataset)
print(bulk_result)

0     0
1     1
2     3
3     0
4     1
     ..
75    3
76    3
77    3
78    3
79    1
Name: label, Length: 80, dtype: int64


In [15]:
image_path = test_dataset.iloc[0]['image']
feature = predictor.predict_feature(image_path)
print(feature)

                                       image_feature  \
0  [0.80504835, 0.5993979, 0.46489826, 0.04663342...   

                                               image  
0  /Users/traveroliver/.gluoncv/datasets/shopee-i...  


In [16]:
test_acc = predictor.evaluate(test_dataset)
print('Top-1 test acc: %.3f' % test_acc['top1'])

[Epoch 1] validation: top1=0.850000 top5=1.000000


Top-1 test acc: 0.850


In [17]:
filename = 'predictor.ag'
predictor.save(filename)
predictor_loaded = ImagePredictor.load(filename)
# use predictor_loaded as usual
result = predictor_loaded.predict(image_path)
print(result)

0    0
Name: label, dtype: int64
